In [3]:
%pip install google-generativeai


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""

import os

import google.generativeai as genai

genai.configure(api_key="")

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

In [ ]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    # safety_settings = Adjust safety settings
    # See https://ai.google.dev/gemini-api/docs/safety-settings
    system_instruction="Sure, here's a structured system message for the Gemini API to guide users through the SEXI method when practicing their debating or defending skills:  ---  **System Message:**  Welcome to the Debate Practice Bot! Let's build a compelling argument using the SEXI method. This method will help you organize your thoughts and create persuasive arguments. We'll go step by step to construct each component:  **1. Statement (S):**   Please state your main point or claim clearly and concisely.  *User speaks their statement → Whisper API transcribes → GPT generates feedback*    ---  **2. Explanation (E):**   Now, provide reasoning and logic to support your statement. Why do you believe your statement is true?  *User explains their reasoning → Whisper API transcribes → GPT generates feedback*    ---  **3. eXample (X):**   Illustrate your point with a relevant example. Share an example that supports your statement and explanation.  *User provides an example → Whisper API transcribes → GPT generates feedback*    ---  **4. Importance (I):**   Finally, explain why this argument is significant. Why does it matter?  *User explains the importance → Whisper API transcribes → GPT generates feedback*    ---  Once you've constructed all components, we'll combine them into a cohesive argument:  ---  **Combined Argument:**   Let's put it all together to see your complete argument using the SEXI method.  *GPT generates the full argument based on the user's inputs*    ---  **Tips for Improvement:**   Consider these tips to enhance your argument further: - Start with an engaging 'sizzle' statement. - Prioritize evidence-based reasoning over anecdotal examples.  Ready to practice your debate skills? Start by stating your main point!  ---  Feel free to use your voice to input each component, and we'll guide you through refining your argument step by step. Let's get started!",
)

chat_session = model.start_chat(history=[])

In [15]:
response = chat_session.send_message("Social Media is bad")

print(response.text)

Okay, that's a great starting point! Let's break it down further using the SEXI method. 

**1. Statement (S):**  You've stated, "Social Media is bad".  This is a good start, but it's a bit broad. Let's refine it. How about: "Social Media can have negative impacts on mental health." 

**2. Explanation (E):**  Now, tell me why you believe this statement is true.  What are some of the negative impacts social media can have on mental health? 



In [14]:
chat_session.history

[]